In [5]:
import csv
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from numpy import nan
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE

from sklearn.utils import resample
plt.style.use('classic')
%matplotlib inline
pd.set_option('display.max_rows', None)

In [6]:
df_train1 = pd.read_csv('sample_data/X_trainData_1.csv')
df_train2 = pd.read_csv('sample_data/Y_trainData_1.csv')
df_test = pd.read_csv('sample_data/X_testData_1.csv')
df_test = df_test.loc[:, df_test.columns != 'ID']

In [7]:
#output = df_train2.drop_duplicates()
#output.groupby('Unnamed: 0').size()
print(df_train1.shape)
print(df_train2.shape)
print(df_test.shape)

(2335, 222)
(3393, 11)
(1000, 222)


In [31]:
res = pd.concat([df_train1, df_train2], axis=1)
res = res[res.OFLX!= -1]
res2=res[res.OFLX==1]
# res3=res[res.OFLX==0]
# df_upsampled = resample(res3,replace=True, n_samples=203,random_state=123)   
# res=pd.concat([res2, df_upsampled], axis=0)
y = res['OFLX']
X = res.drop(['OFLX'], axis = 1) 
X = X.drop(['INH'], axis = 1) 
X = X.drop(['RIF'], axis = 1) 
X = X.drop(['CIP'], axis = 1) 
X = X.drop(['PZA'], axis = 1) 
X = X.drop(['EMB'], axis = 1)
X = X.drop(['CAP'], axis = 1)
X = X.drop(['STR'], axis = 1)
X = X.drop(['AMK'], axis = 1)
X = X.drop(['MOXI'], axis = 1)  
X = X.drop(['KAN'], axis = 1)  

print(y.shape)
print(X.shape)
print(res.shape)
print(res2.shape)
# print(y)
# print(X)
# upsamp = SMOTE(ratio='minority')
# X_sm, y_sm = upsamp.fit_sample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.1)


(690,)
(690, 222)
(690, 233)
(603, 233)


In [66]:
#Best model
# from sklearn.model_selection import GridSearchCV 
# cv_params = {'max_depth': [3,4,5,6], 'min_child_weight': [1,2], 'subsample': [0.9,0.95,1], 'max_delta_step': [0,1,2,3], 'learning_rate':  [0.2, 0.25, 0.3,0.15], 'gamma': [0.05,0.5, 1, 1.5, 2, 5]}  
# gscv = GridSearchCV(xgb.XGBClassifier(n_estimators=600, objective='binary:logistic',
#                     silent=True, nthread=1), cv_params, scoring = 'roc_auc', cv = 5,n_jobs=4)#running grid search cv on model 
# gscv.fit(X_train, y_train)#fitting model
# # gscv.grid_scores_#displaying scores
# best_params= gscv.best_params_
# print(best_params)#use these to train the XGB Classifier
model = xgb.XGBClassifier(learning_rate=0.3,gamma=0,max_depth=6,min_child_weight =1, eval_metric='auc',max_delta_step=1,scale_pos_weight=1,subsample=1,colsample_bytree=1,n_estimators=400, reg_lambda=0.8,reg_alpha=0.3)
model.fit(X_train, y_train)
y_pred = model.predict_proba(df_test)#for converting into probabilities
df = pd.read_csv('Y_testData_1_nolabels_OFLX.csv')
ID = df['ID'].tolist()
print(y_pred.shape)

(1000, 2)


In [67]:
li_id = []
li_val = []
for i in ID:
    # print(i, y_pred[i-1])
    li_id.append(i)
    li_val.append((y_pred[i-1][1]))
csv = pd.DataFrame({'ID': li_id, 'OFLX' :li_val})
csv.to_csv('OFLX-submit.csv', index=False, header=True)  
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))  
print(roc_auc_score(y_test, y_pred))

[[ 2  5]
 [ 3 59]]
              precision    recall  f1-score   support

           0       0.40      0.29      0.33         7
           1       0.92      0.95      0.94        62

    accuracy                           0.88        69
   macro avg       0.66      0.62      0.63        69
weighted avg       0.87      0.88      0.88        69

0.618663594470046
